In [2]:
import pandas as pd
import re
import numpy as np
from pyvi import ViTokenizer, ViPosTagger
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [3]:
data = pd.read_csv('dataPhaseVer2.csv')

In [11]:
data.head()
data = data[['question','target','answer']]
data['target_enc'] = pd.factorize(data.target)[0]

In [13]:
X = data['question']
y = data['target_enc']

In [14]:
# split data into Xtrain, Xtest, ytrain, ytest
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.3,random_state=42)

In [15]:
def transform_row(row):
  
    # Xóa số dòng ở đầu câu
    row = re.sub(r"^[0-9\.]+", "", row)
    
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$", "", row)
    
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("/", " ")
    row = row.lower()
    row = row.strip()
    return row 

In [16]:
def word_segment(word_segment_lib, df_data):
    if word_segment_lib == 'underthesea':
        transformer_result = df_data.apply(lambda text: word_tokenize(text, format='text'))
    else:
        transformer_result = df_data.apply(lambda text: ViTokenizer.tokenize(text))
    return transformer_result

In [17]:
def bag_of_word(train_data):
    vectorizer = CountVectorizer()
    vectorizer = vectorizer.fit(train_data)
    vectorizer_train_data = vectorizer.transform(train_data)
    return vectorizer,vectorizer_train_data

In [18]:
def ifidf_transformer(data):
    tf_idf_model = TfidfTransformer(use_idf=False).fit(data)
    tf_idf_data  = tf_idf_model.transform(data)
    return tf_idf_model,tf_idf_data

In [33]:
def evaluation(clf,tf_idf_test_data,test_data_y):
    # predicted
    y_predicted = clf.predict(tf_idf_test_data)
    
    # accuracy score
    accuracy = accuracy_score(test_data_y,y_predicted)
    
    # precision
    
    classification = classification_report(test_data_y,y_predicted,output_dict=False)
    return y_predicted,accuracy,classification

### Handle to train data

In [34]:
# transform row train data
Xtrain = Xtrain.apply(transform_row)

In [35]:
Xtrain.head(5)

1497    tôi có thể xem trạng thái tài khoản của tôi bằ...
527      mình có thể thanh toán qua thẻ được không hả bạn
914                 khi đăng ký tên miền tôi cần lưu ý gì
221                              chuyển domain qua tenten
113                                                cảm ơn
Name: question, dtype: object

In [36]:
# word_segment data
Xtrain_word_segment = word_segment('pyvi',Xtrain)

In [37]:
# vectorize data 
vectorizer,vectorizer_train_data = bag_of_word(Xtrain_word_segment)

In [38]:
# tfidf vectorize train data: model tfidf and ifidf train
tf_idf_model,tf_idf_data_train = ifidf_transformer(vectorizer_train_data)

## Handle to test data


In [39]:
# transform row train data
Xtest = Xtest.apply(transform_row)

In [40]:
# word segment data
Xtest_word_segment = word_segment('pyvi',Xtest)

In [41]:
# vectorize data
vectorizer_test_data = vectorizer.transform(Xtest_word_segment)

In [42]:
# tfidf vectorize test data: model tfidf and ifidf test
tf_idf_data_test = tf_idf_model.transform(vectorizer_test_data)

## Train model SGD

In [43]:
# Select model and initial model
from sklearn import linear_model
clf = linear_model.SGDClassifier(
                                alpha=0.001,
                                l1_ratio=0,
                                loss='modified_huber',
                                max_iter= 100,
                                random_state=1,
                                tol=0.001
                                  )

In [44]:
# fit model
clf.fit(tf_idf_data_train,ytrain)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0, learning_rate='optimal', loss='modified_huber',
       max_iter=100, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='l2', power_t=0.5, random_state=1, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [45]:
# evaluation model
y_predicted, accuracy, classification = evaluation(clf,tf_idf_data_test,ytest)

In [47]:
print(accuracy)

0.7920792079207921


# I. Create dataframe

In [28]:
# convert Xtrain,Xtest,ytrain,ytest to array
Xtrain_array = np.asarray(Xtrain)
Xtest_array = np.asarray(Xtest)
ytrain_array = np.asarray(ytrain)
ytest_array = np.asarray(ytest)

## 1. Create dataframe Train_X_y and dataset




In [29]:
# create dataframe dataset
dataset = pd.DataFrame({'questionTest':Xtest_array,'true_label':ytest_array, 'predicted_label':y_predicted})

In [30]:
dataset.head()

,questionTest,true_label,predicted_label
0,tên miền,185,185
1,cài os như thế nào vậy,11,11
2,vps của tenten có đặc điểm gì,51,51
3,quên mật khẩu,167,167
4,tôi có thể xem trạng thái tài khoản của tôi bằ...,217,217


In [31]:
# take the values of two columns that difference
dataset['compare'] = np.where(dataset['true_label'] != dataset['predicted_label'],dataset['true_label'],np.nan)

In [32]:
dataset.head()

,questionTest,true_label,predicted_label,compare
0,tên miền,185,185,NaN
1,cài os như thế nào vậy,11,11,NaN
2,vps của tenten có đặc điểm gì,51,51,NaN
3,quên mật khẩu,167,167,NaN
4,tôi có thể xem trạng thái tài khoản của tôi bằ...,217,217,NaN


In [33]:

dataset = dataset.dropna()

In [34]:
# create dataframe Train_X_y
Train_X_y = pd.DataFrame({'Question_Train':Xtrain_array,'true_label':ytrain})

## Merge two dataframe

In [35]:
# Dataframe true
dataframe_true = pd.merge(dataset,Train_X_y,on=['true_label'])

In [36]:
# Arrange values
dataframe_true = dataframe_true.sort_values(by='true_label')

In [37]:
# create dataframe Train_X_y
Train_X_y = pd.DataFrame({'Question_Train':Xtrain_array,'predicted_label':ytrain})

In [44]:
# change column name: 
dataframe_true = dataframe_true.rename(columns={'Question_Train':'Question_Train_True'})
# add a new column
dataframe_true['transformer_questionTest'] = dataframe_true['questionTest'].apply(lambda text: word_tokenize(text, format='text'))

dataframe_true['Transform_Question_Train_True'] = dataframe_true['Question_Train_True'].apply(lambda text: word_tokenize(text,format='text'))
# save to file excel
dataframe_true.to_excel('dataframe_true_SGD.xlsx')

dataframe_true.head()


,questionTest,true_label,predicted_label,compare,Question_Train_True,transformer_questionTest,Transform_Question_Train_True
132,buy domain,6,185,6.0,i want a domain,buy domain,i want a domain
133,buy domain,6,185,6.0,i want a domain for my website,buy domain,i want a domain for my website
426,tenten còn cung cấp thêm về dịch vụ gì nữa ạ,8,221,8.0,bên tenten cung cấp các gói dịch vụ gì,tenten còn cung_cấp thêm về dịch_vụ gì nữa ạ,bên tenten cung_cấp các gói dịch_vụ gì
427,tenten còn cung cấp thêm về dịch vụ gì nữa ạ,8,221,8.0,các loại dịch vụ của tenten,tenten còn cung_cấp thêm về dịch_vụ gì nữa ạ,các loại dịch_vụ của tenten
428,tenten còn cung cấp thêm về dịch vụ gì nữa ạ,8,221,8.0,các gói dịch vụ mà tenten có là gì,tenten còn cung_cấp thêm về dịch_vụ gì nữa ạ,các gói dịch_vụ mà tenten có là gì


In [47]:
# Dataframe predicted
dataframe_predicted = pd.merge(dataset,Train_X_y,on=['predicted_label'])
dataframe_predicted = dataframe_predicted.sort_values(by='true_label')

In [51]:
# change to column name
dataframe_predicted = dataframe_predicted.rename(columns={'Question_Train':'Question_Train_Predicted'})
# add a new column
dataframe_predicted['transformer_questionTest'] = dataframe_predicted['questionTest'].apply(lambda text: word_tokenize(text, format='text'))
dataframe_predicted['Transform_Question_Train_Predicted'] = dataframe_predicted['Question_Train_Predicted'].apply(lambda text: word_tokenize(text,format='text'))

# save to file xlsx
dataframe_predicted.to_excel('dataframe_predicted_SGD3.xlsx')

dataframe_predicted.head()

,questionTest,true_label,predicted_label,compare,Question_Train_Predicted,transformer_questionTest,Transform_Question_Train_Predicted
528,buy domain,6,185,6.0,tôi muốn biết tên miền là gì,buy domain,tôi muốn biết tên miền là gì
521,buy domain,6,185,6.0,tên miền,buy domain,tên miền
523,buy domain,6,185,6.0,tên miền là gì hả bạn,buy domain,tên miền là gì hả bạn
524,buy domain,6,185,6.0,tên miền,buy domain,tên miền
525,buy domain,6,185,6.0,thông tin cần biết về tên miền,buy domain,thông_tin cần biết về tên miền


In [ ]:
dataframe_predicted = dataframe_predicted.sort_values(by='true_label')
dataframe_predicted.to_excel('dataframe_predictedSGD.xlsx')

In [ ]:
dataframe_predicted.head()

In [ ]:
target = data['target_enc'].isin(dataframe_true['true_label'])
target_enc = data[target][['target','target_enc']]
x = target_enc.target.unique()
y = target_enc.target_enc.unique()
target_true_label = pd.DataFrame({'target':x,'target_enc':y})
target_true_label.to_excel('target_true_label.xlsx')
xy.head()

In [ ]:
target = data['target_enc'].isin(dataframe_true['predicted_label'])
target_enc = data[target][['target','target_enc']]
x = target_enc.target.unique()
y = target_enc.target_enc.unique()
target_predicted_label = pd.DataFrame({'target':x,'target_enc':y})

xy.head()

In [ ]:
target_true_label.head()

In [52]:
data.head()

,Unnamed: 0,question,target,answer,target_enc,count
0,0,có mấy bước để thêm domain cho google blog vậy...,add_domain_cho_blog_google,Bước 1: Đăng nhập domain.tenten.vn\n- Add thêm...,0,5
1,1,bạn có thể hướng dẫn giúp mình cách thêm domai...,add_domain_cho_blog_google,Bước 1: Đăng nhập domain.tenten.vn\n- Add thêm...,0,5
2,2,mình phải tự thêm domain google blog hay bạn t...,add_domain_cho_blog_google,Bước 1: Đăng nhập domain.tenten.vn\n- Add thêm...,0,5
3,3,các bước để thêm domain cho blog google như th...,add_domain_cho_blog_google,Bước 1: Đăng nhập domain.tenten.vn\n- Add thêm...,0,5
4,4,Add domain cho blog google,add_domain_cho_blog_google,Bước 1: Đăng nhập domain.tenten.vn\n- Add thêm...,0,5


In [63]:
data[new]

,Unnamed: 0,question,target,answer,target_enc,count
0,0,có mấy bước để thêm domain cho google blog vậy...,add_domain_cho_blog_google,Bước 1: Đăng nhập domain.tenten.vn\n- Add thêm...,0,5
2,2,mình phải tự thêm domain google blog hay bạn t...,add_domain_cho_blog_google,Bước 1: Đăng nhập domain.tenten.vn\n- Add thêm...,0,5
3,3,các bước để thêm domain cho blog google như th...,add_domain_cho_blog_google,Bước 1: Đăng nhập domain.tenten.vn\n- Add thêm...,0,5
5,5,mình vào website nào để add tên miền vào googl...,add_ten_mien_vao_google_site_nhu_the_nao,1. Đăng nhập navi.tenten.vn hoặc domain.tenten...,1,5
6,6,mình phải tự add tên miền vào google site à bạn,add_ten_mien_vao_google_site_nhu_the_nao,1. Đăng nhập navi.tenten.vn hoặc domain.tenten...,1,5
7,7,bạn có thể hướng dẫn mình cách add tên miền và...,add_ten_mien_vao_google_site_nhu_the_nao,1. Đăng nhập navi.tenten.vn hoặc domain.tenten...,1,5
8,8,tôi muốn add tên miền vào google site thì làm ...,add_ten_mien_vao_google_site_nhu_the_nao,1. Đăng nhập navi.tenten.vn hoặc domain.tenten...,1,5
14,14,bạn kiểm tra hộ mình tên miền,ban_kiem_tra_ho_minh_ten_mien,Để liên hệ kinh doanh vui lòng gọi điện thoại ...,2,8
16,16,bạn kiểm tra tên miền của mình với,ban_kiem_tra_ho_minh_ten_mien,Để liên hệ kinh doanh vui lòng gọi điện thoại ...,2,8
19,19,bạn là robot à,ban_ten_gi,Tôi là Robot Chat TENTEN version Beta v.01 :)\...,3,14


In [64]:
Train_X_y

,Question_Train,predicted_label
377,dịch vụ email theo tên miền,62
286,trong trường hợp nào thì mình sử dụng được gói...,43
1473,tôi có thể thuê hosting ở đâu vậy bạn,234
755,bạn có thể giúp mình cách cài đặt lại os được ...,123
126,mình có thể biết thêm về cấu trúc tên miền ở đ...,20
327,bạn có thể đăng kí cloud vps cho mình được không,53
292,bạn có thể giúp mình tăng dung lượng ổ đĩa không,44
1018,hãy hỗ trợ tôi các câu lệnh vps,162
752,root password,122
584,email server dùng như thế nào,89


In [ ]:
user_data_test = input("vui long nhap cau hoi : ")
    


In [ ]:
transform_row_test = transform(user_data_test)